In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as opt
import torch.nn.functional as F
from torchvision.io import read_image
from torchvision import transforms
from torchsummary import summary
from tqdm.notebook import tqdm
from PIL import Image
import optuna
from optuna.trial import TrialState

In [2]:
random.seed(2020)

In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=1)

In [4]:
resize_image = transforms.Resize(size=(227, 227))

In [5]:
def calculate_mean(dataset):
    mean = torch.Tensor([0.,0.,0.])
    
    num_samples = len(dataset)
    for i in range(num_samples):
        image, _, _, _ = dataset[i]
        image = image.type(torch.FloatTensor) / 255
        
        for j in range(3):
            mean[j] += torch.mean(image[j, :, :])
            
    mean = mean / num_samples
    return mean

In [6]:
def subtract_mean(image, mean):
    normed_image = torch.zeros(image.size())
    image = image.type(torch.FloatTensor) / 255
    
    for j in range(3):
        normed_image[j] = image[j, :, :] - mean[j]
    return normed_image

In [7]:
class AdienceDataset(Dataset):
    def __init__(self, annot_file, img_dir, train=False, transform=None):
        self.img_lbls = pd.read_csv(annot_file, header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.is_train = train
        self.mean = torch.Tensor([0.,0.,0.])
    
    def __len__(self):
        return len(self.img_lbls)
    
    def __getitem__(self, idx):
        img_file = self.img_lbls.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_file)
        image = read_image(img_path)
        # image = resize_image(image)
        label = self.img_lbls.iloc[idx, 1]
    
        if self.is_train:
            positive_list = self.img_lbls[self.img_lbls.iloc[:, 1] == label].index.values
            positive_list = np.setdiff1d(positive_list, np.array([idx]))
            positive_item = random.choice(positive_list)
            positive_img = self.img_lbls.iloc[positive_item, 0]
            pos_img_path = os.path.join(self.img_dir, positive_img)
            pos_image = read_image(pos_img_path)
            
            negative_list = self.img_lbls[self.img_lbls.iloc[:, 1] != label].index.values
            negative_item = random.choice(negative_list)
            negative_img = self.img_lbls.iloc[negative_item, 0]
            neg_img_path = os.path.join(self.img_dir, negative_img)
            neg_image = read_image(neg_img_path)

            if self.transform:
                image = self.transform(image, self.mean)
                pos_image = self.transform(pos_image, self.mean)
                neg_image = self.transform(neg_image, self.mean)
                
            return image, pos_image, neg_image, label
        else:
            if self.transform:
                image = self.transform(image, self.mean)

            return image, label
    
    def set_mean(self, mean):
        self.mean = mean

In [8]:
# train_data = AdienceDataset("../train.csv", "../cropped_Adience/", train=True)
train_data = AdienceDataset("../train.csv", "../cropped_Adience/", train=True, transform=subtract_mean)
test_data = AdienceDataset("../test.csv", "../cropped_Adience/", train=True, transform=subtract_mean)
val_data = AdienceDataset("../val.csv", "../cropped_Adience/", train=True, transform=subtract_mean)

In [9]:
# mean = calculate_mean(train_data)
mean = torch.Tensor([0.5062, 0.3581, 0.3099])
mean

tensor([0.5062, 0.3581, 0.3099])

In [10]:
train_data.set_mean(mean)
test_data.set_mean(mean)
val_data.set_mean(mean)

In [11]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, anchor, positive, negative):
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()

In [12]:
class CombinedLoss(nn.Module):
    def __init__(self, alpha=1.0, beta=1.0):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.triplet = TripletLoss(margin=1.0)
        self.classification = nn.CrossEntropyLoss()
        
    def forward(self, anchor, positive, negative, classification_out, labels):
        triplet_loss = self.triplet(anchor, positive, negative)
        classification_loss = self.classification(classification_out, labels)
        total_loss = (self.alpha * triplet_loss) + (self.beta * classification_loss)
        
        return total_loss

In [13]:
class Net(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(num_channels, 96, kernel_size=7, stride=4)
        self.conv11 = nn.Conv2d(96, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(256, 256, kernel_size=5, padding=2)
        self.conv33 = nn.Conv2d(256, 384, kernel_size=1)
        self.conv4 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        
        self.fc6 = nn.Linear(in_features=13824, out_features=512)
        self.fc7 = nn.Linear(in_features=512, out_features=512)
        self.fc8 = nn.Linear(in_features=512, out_features=num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x, alpha: float):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        sl1 = self.conv11(x)
        sl1 = self.relu(sl1)
        
        x = self.conv2(x)
        x = self.relu((1-alpha)*x + alpha*sl1)
        # x = self.relu(x + sl1)
        # x = self.relu(x)
        sl2 = x
        
        x = self.conv3(x)
        x = self.relu((1-alpha)*x + alpha*sl2)
        # x = self.relu(x + sl2)
        # x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        sl3 = self.conv33(x)
        sl3 = self.relu(sl3)
        
        x = self.conv4(x)
        x = self.relu((1-alpha)*x + alpha*sl3)
        # x = self.relu(x + sl3)
        # x = self.relu(x)
        sl4 = x
        
        x = self.conv5(x)
        x = self.relu((1-alpha)*x + alpha*sl4)
        # x = self.relu(x + sl4)
        # x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc7(x)
        x_triplet = x
        x = self.relu(x)
        x = self.dropout(x)
        
        x_classification = self.fc8(x)
        
        return x_triplet, x_classification

In [14]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)

In [16]:
best_accu = 0.9238635897636414
def objective(trial):
    model = Net(3, 2)
    model.apply(init_weights)
    model = torch.jit.script(model).to(device)
    
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    mom = trial.suggest_float('momentum', 0.7, 0.99)
    wd = trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True)
    optimizer = opt.SGD(model.parameters(), lr=lr, momentum=mom, weight_decay=wd)
    criterion = torch.jit.script(CombinedLoss())
    
    alpha = trial.suggest_float("alpha", 0.1, 0.5, step=0.1)
    batch_size = trial.suggest_int('batch_size', 10, 100)
    num_epochs = trial.suggest_int('epochs', 10, 100)
    
    print("Learning rate: "+ str(lr))
    print("Momentum: "+ str(mom))
    print("Weight decay: "+ str(wd))
    print("Alpha: "+ str(alpha))
    print("Batch size: "+ str(batch_size))
    print("Number of epochs: "+ str(num_epochs))
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
        
        # train(model, optim, train_loader)
        # training loop
        running_loss = []
        running_accu = []
        
        model.train()
        for step, (img, pos_img, neg_img, label) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
            img, pos_img, neg_img, label = img.to(device), pos_img.to(device), neg_img.to(device), label.to(device)

            optimizer.zero_grad()
            anchor, output = model(img, alpha)
            pos, _ = model(pos_img, alpha)
            neg, _ = model(neg_img, alpha)
            
            pred = torch.argmax(output, 1)
            accuracy = torch.eq(pred, label).sum() / len(img)

            loss = criterion(anchor, pos, neg, output, label)
            loss.backward()
            optimizer.step()

            running_accu.append(accuracy.cpu().detach().numpy())
            running_loss.append(loss.cpu().detach().numpy())
        print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))
        
        # accuracy = test(model, val_loader)
        # validation loop
        val_loss = []
        val_accu = []

        model.eval()
        with torch.no_grad():
            for img, pos_img, neg_img, label in tqdm(val_loader):
                img, pos_img, neg_img, label = img.to(device), pos_img.to(device), neg_img.to(device), label.to(device)
                
                anchor, output = model(img, alpha)
                pos, _ = model(pos_img, alpha)
                neg, _ = model(neg_img, alpha)
                
                pred = torch.argmax(output, 1)
                accuracy = torch.eq(pred, label).sum() / len(img)
                
                loss = criterion(anchor, pos, neg, output, label)
                
                val_accu.append(accuracy.cpu().detach().numpy())
                val_loss.append(loss.cpu().detach().numpy())
        val_accu = np.mean(val_accu)
        val_loss = np.mean(val_loss)
        print("Val Loss: {:.4f} - Val Accuracy: {:.4f}".format(val_loss, val_accu))
        
        trial.report(val_accu, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    global best_accu
    if val_accu > best_accu:
        best_accu = val_accu
        print("Saving best model...")
        torch.save(model.state_dict(), "../incr_skip_triplet_00.pt")
            
    return val_accu

In [17]:
study = optuna.create_study(direction='maximize',
                            study_name='incremented-skip-alpha-triplet-libfaceid-study',
                            storage='sqlite:///study.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=1)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

# Display the study statistics
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

[I 2023-11-19 16:10:55,240] Using an existing study with name 'incremented-skip-alpha-triplet-libfaceid-study' instead of creating a new one.


Learning rate: 0.0003351653872818826
Momentum: 0.7715726842658507
Weight decay: 0.0017397239196054414
Alpha: 0.1
Batch size: 49
Number of epochs: 81


Epochs:   0%|          | 0/81 [00:00<?, ?it/s]

Training:   0%|          | 0/286 [00:00<?, ?it/s]

Epoch: 1/81 - Loss: 1.6994 - Accuracy: 0.5334


...

Training:   0%|          | 0/286 [00:00<?, ?it/s]

Epoch: 81/81 - Loss: 0.0357 - Accuracy: 0.9987


  0%|          | 0/36 [00:00<?, ?it/s]

[I 2023-11-19 17:46:42,211] Trial 5 finished with value: 0.9082199931144714 and parameters: {'learning_rate': 0.0003351653872818826, 'momentum': 0.7715726842658507, 'weight_decay': 0.0017397239196054414, 'alpha': 0.1, 'batch_size': 49, 'epochs': 81}. Best is trial 3 with value: 0.9238635897636414.


Val Loss: 0.8156 - Val Accuracy: 0.9082

Study statistics: 
  Number of finished trials:  6
  Number of pruned trials:  0
  Number of complete trials:  5


In [18]:
study = optuna.create_study(direction='maximize',
                            study_name='incremented-skip-alpha-triplet-libfaceid-study',
                            storage='sqlite:///study.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=5)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

# Display the study statistics
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

[I 2023-11-19 17:46:42,302] Using an existing study with name 'incremented-skip-alpha-triplet-libfaceid-study' instead of creating a new one.


Learning rate: 0.009585726485878144
Momentum: 0.7294728974244998
Weight decay: 0.007569151600620028
Alpha: 0.2
Batch size: 18
Number of epochs: 62


Epochs:   0%|          | 0/62 [00:00<?, ?it/s]

Training:   0%|          | 0/777 [00:00<?, ?it/s]

Epoch: 1/62 - Loss: 1.6979 - Accuracy: 0.5351


...

Training:   0%|          | 0/518 [00:00<?, ?it/s]

Epoch: 1/50 - Loss: 1.7067 - Accuracy: 0.4745


  0%|          | 0/65 [00:00<?, ?it/s]

[I 2023-11-19 18:13:23,535] Trial 10 pruned. 


Val Loss: 1.6892 - Val Accuracy: 0.4658

Study statistics: 
  Number of finished trials:  11
  Number of pruned trials:  5
  Number of complete trials:  5


In [19]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  0.9238635897636414
  Params: 
    alpha: 0.1
    batch_size: 22
    learning_rate: 0.00027039716003689977
    momentum: 0.8456179921920867
    weight_decay: 1.3141490080199303e-05


In [ ]:
# base model

Best trial:
Value:  0.9344444870948792
Params: 
batch_size: 60
learning_rate: 0.009437359045100662
momentum: 0.7021887822156345
weight_decay: 5.982011475092227e-05

In [ ]:
# skip model

Best trial:
Value:  0.9204047918319702
Params: 
batch_size: 39
learning_rate: 0.0003622895790391199
momentum: 0.8296680142263324
weight_decay: 0.004568653389880783

In [ ]:
# skip model (alpha=0.1)

Best trial:
Value:  0.9238635897636414
Params:
batch_size: 22
learning_rate: 0.00027039716003689977
momentum: 0.8456179921920867
weight_decay: 1.3141490080199303e-05